# Creacion de DataFrame a partir de RDD

In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField 
from pyspark.sql.types import IntegerType, StringType, FloatType
from pyspark.sql.types import Row

from pyspark.sql import SQLContext

In [2]:
spark = SparkContext(master='local', appName="TransformacionesYAcciones")
sqlContext = SQLContext(spark)

24/01/18 16:35:33 WARN Utils: Your hostname, carloscastau resolves to a loopback address: 127.0.1.1; using 192.168.0.13 instead (on interface enp3s0)
24/01/18 16:35:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/18 16:35:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/home/carloscastau/Documents/platzi/data_engineer/curso-apache-spark-platzi-master/venv/lib/python3.10/site-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
pwd

'/home/carloscastau/Documents/platzi/data_engineer/curso-apache-spark-platzi-master'

In [4]:
!ls ./files/

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [5]:
path = './files/'
deportistaOlimpicoRDD = spark.textFile(path+'deportista.csv') \
    .map(lambda line: line.split(","))

deportistaOlimpicoRDD2 = spark.textFile(path+'deportista2.csv') \
    .map(lambda line: line.split(","))

deportistaOlimpicoRDD = deportistaOlimpicoRDD.union(deportistaOlimpicoRDD2)

In [6]:
deportistaOlimpicoRDD.take(5)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278']]

In [7]:
# Retiramos encabezado
def eliminarEncabezado(indice, iterador):
    return iter(list(iterador)[1:])

In [8]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.\
        mapPartitionsWithIndex(eliminarEncabezado)

In [9]:
deportistaOlimpicoRDD.take(5)

[['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278'],
 ['5', 'Christine Jacoba Aaftink', '2', '21', '185', '82', '705']]

In [10]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.map(lambda l: (
    int(l[0]),
    l[1],
    int(l[2]),
    int(l[3]),
    int(l[4]),
    float(l[5]),
    int(l[6])
))

In [11]:
deportistaOlimpicoRDD.take(5)

[(1, 'A Dijiang', 1, 24, 180, 80.0, 199),
 (2, 'A Lamusi', 1, 23, 170, 60.0, 199),
 (3, 'Gunnar Nielsen Aaby', 1, 24, 0, 0.0, 273),
 (4, 'Edgar Lindenau Aabye', 1, 34, 0, 0.0, 278),
 (5, 'Christine Jacoba Aaftink', 2, 21, 185, 82.0, 705)]

In [12]:
schema = StructType([
    StructField('deportista_id', IntegerType(), False),
    StructField('nombre', StringType(), False),
    StructField('genero', IntegerType(), False),
    StructField('edad', IntegerType(), False),
    StructField('altura', IntegerType(), False),
    StructField('peso', FloatType(), False),
    StructField('equipo_id', IntegerType(), False)
])

In [13]:
deportistaDF = sqlContext.createDataFrame(deportistaOlimpicoRDD, schema)

In [14]:
deportistaDF.show(5)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|80.0|      199|
|            2|            A Lamusi|     1|  23|   170|60.0|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|
|            4|Edgar Lindenau Aabye|     1|  34|     0| 0.0|      278|
|            5|Christine Jacoba ...|     2|  21|   185|82.0|      705|
+-------------+--------------------+------+----+------+----+---------+
only showing top 5 rows



## Reto crear los dataframes restantes

In [15]:
# Paises Equipos
equiposOlimpicosRDD = spark.textFile(path+'paises.csv') \
    .map(lambda line: line.split(",")) \
    .mapPartitionsWithIndex(eliminarEncabezado) \
    .map(lambda l: (
    int(l[0]),
    l[1],
    l[2]
    ))

equiposSchema = StructType([
    StructField('id', IntegerType(), False),
    StructField('equipo', StringType(), False),
    StructField('sigla', StringType(), False),
])

paisesDF = sqlContext.createDataFrame(equiposOlimpicosRDD, equiposSchema)

paisesDF.show(5)

+---+--------------------+-----+
| id|              equipo|sigla|
+---+--------------------+-----+
|  1|         30. Februar|  AUT|
|  2|A North American ...|  MEX|
|  3|           Acipactli|  MEX|
|  4|             Acturus|  ARG|
|  5|         Afghanistan|  AFG|
+---+--------------------+-----+
only showing top 5 rows



In [16]:
# Deportes
deporteSchema = StructType([
    StructField('deporte_id', IntegerType(), False),
    StructField('deporte', StringType(), False)
])

# Paso 2 Crea DataFrame con la lectura del Schema
deportesDF = sqlContext.read.schema(deporteSchema) \
            .option('header', 'true').csv(path+'deporte.csv')

deportesDF.show(5)

+----------+-------------+
|deporte_id|      deporte|
+----------+-------------+
|         1|   Basketball|
|         2|         Judo|
|         3|     Football|
|         4|   Tug-Of-War|
|         5|Speed Skating|
+----------+-------------+
only showing top 5 rows



In [17]:
# Evento
eventoSchema = StructType([
    StructField('evento_id', IntegerType(), False),
    StructField('nombre', StringType(), False),
    StructField('deporte_id', IntegerType(), False),
    
])

# Paso 2 Crea DataFrame con la lectura del Schema
deportesOlimpicosDF = sqlContext.read.schema(eventoSchema) \
            .option('header', 'true').csv(path+'evento.csv')

deportesOlimpicosDF.show(5)

+---------+--------------------+----------+
|evento_id|              nombre|deporte_id|
+---------+--------------------+----------+
|        1|Basketball Men's ...|         1|
|        2|Judo Men's Extra-...|         2|
|        3|Football Men's Fo...|         3|
|        4|Tug-Of-War Men's ...|         4|
|        5|Speed Skating Wom...|         5|
+---------+--------------------+----------+
only showing top 5 rows



24/01/18 16:35:53 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: evento_id, evento, deporte_id
 Schema: evento_id, nombre, deporte_id
Expected: nombre but found: evento
CSV file: file:///home/carloscastau/Documents/platzi/data_engineer/curso-apache-spark-platzi-master/files/evento.csv


In [18]:
# resultados
resultadosSchema = StructType([
    StructField('resultado_id', IntegerType(), False),
    StructField('medalla', StringType(), False),
    StructField('deportista_id', IntegerType(), False),
    StructField('juego_id', IntegerType(), False),
    StructField('evento_id', IntegerType(), False),
    
])

# Paso 2 Crea DataFrame con la lectura del Schema
resultadoDF = sqlContext.read.schema(resultadosSchema) \
            .option('header', 'true').csv(path+'resultados.csv')

resultadoDF.show(5)

+------------+-------+-------------+--------+---------+
|resultado_id|medalla|deportista_id|juego_id|evento_id|
+------------+-------+-------------+--------+---------+
|           1|     NA|            1|      39|        1|
|           2|     NA|            2|      49|        2|
|           3|     NA|            3|       7|        3|
|           4|   Gold|            4|       2|        4|
|           5|     NA|            5|      36|        5|
+------------+-------+-------------+--------+---------+
only showing top 5 rows



In [19]:
# juegos
juegoSchema = StructType([
    StructField('juego_id', IntegerType(), False),
    StructField('anio', StringType(), False),
    StructField('temporada', StringType(), False),
    StructField('ciudad', StringType(), False)
])

# Paso 2 Crea DataFrame con la lectura del Schema
juegoDF = sqlContext.read.schema(juegoSchema) \
            .option('header', 'true').csv(path+'juegos.csv')

juegoDF.show(5)

24/01/18 16:35:54 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 5, schema size: 4
CSV file: file:///home/carloscastau/Documents/platzi/data_engineer/curso-apache-spark-platzi-master/files/juegos.csv


+--------+-----------+---------+------+
|juego_id|       anio|temporada|ciudad|
+--------+-----------+---------+------+
|       1|1896 Verano|     1896|Verano|
|       2|1900 Verano|     1900|Verano|
|       3|1904 Verano|     1904|Verano|
|       4|1906 Verano|     1906|Verano|
|       5|1908 Verano|     1908|Verano|
+--------+-----------+---------+------+
only showing top 5 rows



## Operaciones sobre DF


In [20]:
#ver el Schema
deportesDF.printSchema()

root
 |-- deporte_id: integer (nullable = true)
 |-- deporte: string (nullable = true)



In [21]:
deportistaDF.printSchema()

root
 |-- deportista_id: integer (nullable = false)
 |-- nombre: string (nullable = false)
 |-- genero: integer (nullable = false)
 |-- edad: integer (nullable = false)
 |-- altura: integer (nullable = false)
 |-- peso: float (nullable = false)
 |-- equipo_id: integer (nullable = false)



In [22]:
# renombrado de columnas
deportistaOlimpicoDF = deportistaDF \
    .withColumnRenamed("genero", 'sexo') \
    .drop('altura')

In [23]:
deportistaOlimpicoDF.printSchema()

root
 |-- deportista_id: integer (nullable = false)
 |-- nombre: string (nullable = false)
 |-- sexo: integer (nullable = false)
 |-- edad: integer (nullable = false)
 |-- peso: float (nullable = false)
 |-- equipo_id: integer (nullable = false)



In [24]:
# Usando el rey SELECT
from pyspark.sql.functions import *

deportistaOlimpicoDF = deportistaOlimpicoDF.select("deportista_id", "nombre",
                           col("edad").alias("edadAlJugar"),
                           "equipo_id"
                          )

In [25]:
deportistaOlimpicoDF.show(5)

+-------------+--------------------+-----------+---------+
|deportista_id|              nombre|edadAlJugar|equipo_id|
+-------------+--------------------+-----------+---------+
|            1|           A Dijiang|         24|      199|
|            2|            A Lamusi|         23|      199|
|            3| Gunnar Nielsen Aaby|         24|      273|
|            4|Edgar Lindenau Aabye|         34|      278|
|            5|Christine Jacoba ...|         21|      705|
+-------------+--------------------+-----------+---------+
only showing top 5 rows



In [26]:
# Usando filer
deportistaOlimpicoDF = deportistaOlimpicoDF.filter(
                        (deportistaOlimpicoDF.edadAlJugar != 0)
                    )

In [27]:
deportistaOlimpicoDF.sort("edadAlJugar").show()

+-------------+--------------------+-----------+---------+
|deportista_id|              nombre|edadAlJugar|equipo_id|
+-------------+--------------------+-----------+---------+
|        71691|  Dimitrios Loundras|         10|      333|
|        22411|Magdalena Cecilia...|         11|      413|
|        70616|          Liu Luyang|         11|      199|
|        37333|Carlos Bienvenido...|         11|      982|
|        76675|   Marcelle Matthews|         11|      967|
|        40129|    Luigina Giavotti|         11|      507|
|       118925|Megan Olwen Deven...|         11|      413|
|        47618|Sonja Henie Toppi...|         11|      742|
|       126307|        Liana Vicens|         11|      825|
|        51268|      Beatrice Hutiu|         11|      861|
|        52070|        Etsuko Inada|         11|      514|
|        72854|      Licia Macchini|         12|      507|
|         5291|Marcia Arriaga La...|         12|      656|
|        74712|     Carla Marangoni|         12|      50

## Agrupaciones y operaciones join sobre DF

In [28]:
#visualizamos los Schemas de los DataFrames
deportistaOlimpicoDF.printSchema()

root
 |-- deportista_id: integer (nullable = false)
 |-- nombre: string (nullable = false)
 |-- edadAlJugar: integer (nullable = false)
 |-- equipo_id: integer (nullable = false)



In [29]:
resultadoDF.printSchema()

root
 |-- resultado_id: integer (nullable = true)
 |-- medalla: string (nullable = true)
 |-- deportista_id: integer (nullable = true)
 |-- juego_id: integer (nullable = true)
 |-- evento_id: integer (nullable = true)



In [30]:
juegoDF.printSchema()

root
 |-- juego_id: integer (nullable = true)
 |-- anio: string (nullable = true)
 |-- temporada: string (nullable = true)
 |-- ciudad: string (nullable = true)



In [31]:
deportesOlimpicosDF.printSchema()

root
 |-- evento_id: integer (nullable = true)
 |-- nombre: string (nullable = true)
 |-- deporte_id: integer (nullable = true)



In [32]:
paisesDF.printSchema()

root
 |-- id: integer (nullable = false)
 |-- equipo: string (nullable = false)
 |-- sigla: string (nullable = false)



In [33]:
# JOINS
deportistaOlimpicoDF \
    .join(
    resultadoDF,
    deportistaOlimpicoDF.deportista_id == resultadoDF.deportista_id,
    "left"
    ) \
    .join(
    juegoDF,
    juegoDF.juego_id == resultadoDF.juego_id,
    "left"
    ) \
    .join(
    deportesOlimpicosDF,
    deportesOlimpicosDF.evento_id == resultadoDF.evento_id,
    "left"
    ) \
    .select(deportistaOlimpicoDF.nombre, 
            "edadAlJugar",
            "medalla", 
            col("anio").alias("Anio de Juego"),
            deportesOlimpicosDF.nombre.alias("Nombre de disciplina")
           ).show()


24/01/18 16:35:59 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: evento_id, evento
 Schema: evento_id, nombre
Expected: nombre but found: evento
CSV file: file:///home/carloscastau/Documents/platzi/data_engineer/curso-apache-spark-platzi-master/files/evento.csv
24/01/18 16:35:59 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , nombre_juego
 Schema: juego_id, anio
Expected: juego_id but found: 
CSV file: file:///home/carloscastau/Documents/platzi/data_engineer/curso-apache-spark-platzi-master/files/juegos.csv


+--------------------+-----------+-------+-------------+--------------------+
|              nombre|edadAlJugar|medalla|Anio de Juego|Nombre de disciplina|
+--------------------+-----------+-------+-------------+--------------------+
|           A Dijiang|         24|     NA|  1992 Verano|Basketball Men's ...|
|            A Lamusi|         23|     NA|  2012 Verano|Judo Men's Extra-...|
| Gunnar Nielsen Aaby|         24|     NA|  1920 Verano|Football Men's Fo...|
|Edgar Lindenau Aabye|         34|   Gold|  1900 Verano|Tug-Of-War Men's ...|
|Christine Jacoba ...|         21|     NA|1994 Invierno|Speed Skating Wom...|
|Christine Jacoba ...|         21|     NA|1994 Invierno|Speed Skating Wom...|
|Christine Jacoba ...|         21|     NA|1992 Invierno|Speed Skating Wom...|
|Christine Jacoba ...|         21|     NA|1992 Invierno|Speed Skating Wom...|
|Christine Jacoba ...|         21|     NA|1988 Invierno|Speed Skating Wom...|
|Christine Jacoba ...|         21|     NA|1988 Invierno|Speed Sk

### Reto

Hacer un join con todas las medallas ganadoras unidas con el pais y el equipo al que pertenecen estas medallas.

In [34]:
resultadoDF \
    .join(
    deportistaOlimpicoDF,
    resultadoDF.deportista_id == deportistaOlimpicoDF.deportista_id,
    "left"
    ) \
    .join(
    paisesDF,
    deportistaOlimpicoDF.equipo_id == paisesDF.id,
    "left"    
    ) \
    .select("medalla",
            paisesDF.equipo,
            paisesDF.sigla
    ) \
    .where(resultadoDF.medalla != "NA") .show()

+-------+--------------+-----+
|medalla|        equipo|sigla|
+-------+--------------+-----+
| Bronze|       Finland|  FIN|
| Bronze|       Finland|  FIN|
| Bronze|       Finland|  FIN|
| Bronze|       Finland|  FIN|
|   Gold|       Finland|  FIN|
|   Gold|       Finland|  FIN|
|   Gold|       Finland|  FIN|
| Bronze|       Finland|  FIN|
|   Gold|        Norway|  NOR|
| Bronze|        Norway|  NOR|
| Silver|        Norway|  NOR|
| Bronze|        Norway|  NOR|
| Silver|        Norway|  NOR|
|   Gold|        Norway|  NOR|
|   Gold|        Norway|  NOR|
|   Gold|        Norway|  NOR|
|   Gold|        Norway|  NOR|
| Silver|        Norway|  NOR|
|   Gold|Denmark/Sweden|  SWE|
| Bronze|   Netherlands|  NED|
+-------+--------------+-----+
only showing top 20 rows



## Solución reto joins
Solucion del profesor

In [35]:
resultadoDF.filter(resultadoDF.medalla != "NA") \
    .join(
        deportistaOlimpicoDF,
        resultadoDF.deportista_id == deportistaOlimpicoDF.deportista_id,
        "left"
    ) \
    .join(
        paisesDF,
        deportistaOlimpicoDF.equipo_id == paisesDF.id,
        "left"    
    ) \
    .select("medalla", paisesDF.equipo, paisesDF.sigla) \
    .sort( col("sigla").desc()) \
    .show()

+-------+--------+-----+
|medalla|  equipo|sigla|
+-------+--------+-----+
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Bronze|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
+-------+--------+-----+
only showing top 20 rows



## Funciones de agrupación


In [36]:
medallistaXAnio = deportistaOlimpicoDF \
    .join(
        resultadoDF, 
        deportistaOlimpicoDF.deportista_id == resultadoDF.deportista_id, 
        "left"
    ) \
    .join(
        juegoDF, 
        juegoDF.juego_id == resultadoDF.juego_id, 
        "left"
    ) \
    .join(
        paisesDF, 
        deportistaOlimpicoDF.equipo_id == paisesDF.id, 
        "left"
    ) \
    .join(
        deportesOlimpicosDF, 
        deportesOlimpicosDF.evento_id == resultadoDF.evento_id, 
        "left"
    ) \
    .join(
        deportesDF, 
        deportesOlimpicosDF.deporte_id == deportesDF.deporte_id, 
        "left"
    ) \
    .select(
        "sigla",
        "anio",
        "medalla",
        deportesOlimpicosDF.nombre.alias("Nombre subdisciplina"),
        deportesDF.deporte.alias("Nombre disciplina"),
        deportistaOlimpicoDF.nombre    
    )

In [37]:
medallistaXAnio.show()

24/01/18 16:36:11 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: evento_id, evento, deporte_id
 Schema: evento_id, nombre, deporte_id
Expected: nombre but found: evento
CSV file: file:///home/carloscastau/Documents/platzi/data_engineer/curso-apache-spark-platzi-master/files/evento.csv
24/01/18 16:36:11 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , nombre_juego
 Schema: juego_id, anio
Expected: juego_id but found: 
CSV file: file:///home/carloscastau/Documents/platzi/data_engineer/curso-apache-spark-platzi-master/files/juegos.csv


+-----+-------------+-------+--------------------+--------------------+--------------------+
|sigla|         anio|medalla|Nombre subdisciplina|   Nombre disciplina|              nombre|
+-----+-------------+-------+--------------------+--------------------+--------------------+
|  CHN|  1992 Verano|     NA|Basketball Men's ...|          Basketball|           A Dijiang|
|  CHN|  2012 Verano|     NA|Judo Men's Extra-...|                Judo|            A Lamusi|
|  DEN|  1920 Verano|     NA|Football Men's Fo...|            Football| Gunnar Nielsen Aaby|
|  SWE|  1900 Verano|   Gold|Tug-Of-War Men's ...|          Tug-Of-War|Edgar Lindenau Aabye|
|  NED|1994 Invierno|     NA|Speed Skating Wom...|       Speed Skating|Christine Jacoba ...|
|  NED|1994 Invierno|     NA|Speed Skating Wom...|       Speed Skating|Christine Jacoba ...|
|  NED|1992 Invierno|     NA|Speed Skating Wom...|       Speed Skating|Christine Jacoba ...|
|  NED|1992 Invierno|     NA|Speed Skating Wom...|       Speed Skating

In [38]:
# descartamos deportistas sin medalla
medallistaXAnio2 = medallistaXAnio.filter(resultadoDF.medalla != "NA") \
                    .sort("anio") \
                    .groupBy("sigla", "anio", "Nombre subdisciplina") \
                    .count()

medallistaXAnio2.show()

24/01/18 16:36:13 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: evento_id, evento, deporte_id
 Schema: evento_id, nombre, deporte_id
Expected: nombre but found: evento
CSV file: file:///home/carloscastau/Documents/platzi/data_engineer/curso-apache-spark-platzi-master/files/evento.csv
24/01/18 16:36:13 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , nombre_juego
 Schema: juego_id, anio
Expected: juego_id but found: 
CSV file: file:///home/carloscastau/Documents/platzi/data_engineer/curso-apache-spark-platzi-master/files/juegos.csv


+-----+-------------+--------------------+-----+
|sigla|         anio|Nombre subdisciplina|count|
+-----+-------------+--------------------+-----+
|  KOR|2006 Invierno|Short Track Speed...|    2|
|  RUS|  2004 Verano|Wrestling Men's S...|    1|
|  USA|  2004 Verano|Synchronized Swim...|    8|
|  RUS|  2016 Verano|Shooting Women's ...|    1|
|  RUS|  2008 Verano|Wrestling Men's L...|    1|
|  SWE|  1936 Verano|Art Competitions ...|    1|
|  EUN|2002 Invierno|Cross Country Ski...|    1|
|  GBR|  1956 Verano|Sailing Mixed 5.5...|    4|
|  USA|  1988 Verano|Volleyball Men's ...|   12|
|  FRA|  1932 Verano|Cycling Men's Tea...|    4|
|  USA|  1996 Verano|Equestrianism Mix...|    4|
|  USA|  1996 Verano|Athletics Men's H...|    1|
|  GBR|  2008 Verano|Boxing Men's Midd...|    1|
|  FRA|  1984 Verano|Fencing Men's Sab...|    5|
|  USA|  2012 Verano|Gymnastics Women'...|    1|
|  FRA|  2004 Verano|Rowing Men's Ligh...|    2|
|  NOR|1932 Invierno|Speed Skating Men...|    1|
|  FRG|  1992 Verano

In [39]:
medallistaXAnio2.printSchema()

root
 |-- sigla: string (nullable = true)
 |-- anio: string (nullable = true)
 |-- Nombre subdisciplina: string (nullable = true)
 |-- count: long (nullable = false)



In [40]:
medallistaXAnio2.groupBy("Sigla", "anio") \
    .agg(sum("count").alias("Total de medallas"), \
        avg("count").alias("Medallas promedio")).show()

# .agg() o agregate es la forma recomendada por la documentacion.

24/01/18 16:36:18 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , nombre_juego
 Schema: juego_id, anio
Expected: juego_id but found: 
CSV file: file:///home/carloscastau/Documents/platzi/data_engineer/curso-apache-spark-platzi-master/files/juegos.csv
24/01/18 16:36:18 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: evento_id, evento, deporte_id
 Schema: evento_id, nombre, deporte_id
Expected: nombre but found: evento
CSV file: file:///home/carloscastau/Documents/platzi/data_engineer/curso-apache-spark-platzi-master/files/evento.csv


+-----+-------------+-----------------+------------------+
|Sigla|         anio|Total de medallas| Medallas promedio|
+-----+-------------+-----------------+------------------+
|  SWE|  1976 Verano|               10|               2.0|
|  MGL|  2008 Verano|                5|              1.25|
|  SUI|2014 Invierno|               29|3.2222222222222223|
|  ETH|  2004 Verano|                7|              1.75|
|  BEL|  2000 Verano|                7|               1.4|
|  AUT|  1928 Verano|                5|              1.25|
|  SYR|  1984 Verano|                1|               1.0|
|  NED|1992 Invierno|                4|1.3333333333333333|
|  MAS|  2012 Verano|                2|               1.0|
|  ITA|  1996 Verano|               69| 2.225806451612903|
|  THA|  2008 Verano|                4|               1.0|
|  URS|1984 Invierno|               56|               2.8|
|  DEN|  1896 Verano|                6|               1.0|
|  GRN|  2016 Verano|                1|               1.

# SQL

In [41]:
# registro de DataFrames como tablas SQL
resultadoDF.registerTempTable("resultado")
deportistaOlimpicoDF.registerTempTable("deportista")
paisesDF.registerTempTable("paises")

/home/carloscastau/Documents/platzi/data_engineer/curso-apache-spark-platzi-master/venv/lib/python3.10/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [42]:
sqlContext.sql("SELECT * FROM deportista").show(5)

+-------------+--------------------+-----------+---------+
|deportista_id|              nombre|edadAlJugar|equipo_id|
+-------------+--------------------+-----------+---------+
|            1|           A Dijiang|         24|      199|
|            2|            A Lamusi|         23|      199|
|            3| Gunnar Nielsen Aaby|         24|      273|
|            4|Edgar Lindenau Aabye|         34|      278|
|            5|Christine Jacoba ...|         21|      705|
+-------------+--------------------+-----------+---------+
only showing top 5 rows



In [43]:
sqlContext.sql("""
                SELECT DISTINCT medalla, equipo, sigla FROM resultado as r
                JOIN deportista as d
                ON r.deportista_id = d.deportista_id
                JOIN paises as p
                ON p.id = d.equipo_id
                WHERE medalla <> "NA"
                ORDER BY sigla DESC
                """).show(15)

+-------+--------------------+-----+
|medalla|              equipo|sigla|
+-------+--------------------+-----+
| Bronze|            Zimbabwe|  ZIM|
| Silver|            Zimbabwe|  ZIM|
|   Gold|            Zimbabwe|  ZIM|
| Silver|              Zambia|  ZAM|
|   Gold|          Yugoslavia|  YUG|
| Silver|          Yugoslavia|  YUG|
| Bronze|          Yugoslavia|  YUG|
| Bronze|West Indies Feder...|  WIF|
| Silver|             Vietnam|  VIE|
|   Gold|             Vietnam|  VIE|
| Bronze|           Venezuela|  VEN|
| Silver|           Venezuela|  VEN|
|   Gold|           Venezuela|  VEN|
|   Gold|          Uzbekistan|  UZB|
| Silver|          Uzbekistan|  UZB|
+-------+--------------------+-----+
only showing top 15 rows



In [44]:
spark

<SparkContext master=local appName=TransformacionesYAcciones>

# UDF

In [45]:
!ls ./files/

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [46]:
!head -n 5 ./files/deportistaError.csv

deportista_id,nombre,genero,edad,altura,peso,equipo_id
1,A Dijiang,1,24,180,80,199
2,A Lamusi,1,23,170,60,199
3,Gunnar Nielsen Aaby,1,24,,,273
4,Edgar Lindenau Aabye,1,34,,,278


In [47]:
DeportistaError_schema = StructType([
    StructField("deportista_id", StringType(), False),
    StructField("nombre", StringType(), False),
    StructField("genero", StringType(), False),
    StructField("edad", StringType(), False),
    StructField("altura", StringType(), False), 
    StructField("peso", StringType(), False),
    StructField("equipo_id", StringType(), False)
])

deportistaErrorDF = sqlContext.read.format("csv").\
            option("header", True).\
            schema(DeportistaError_schema).\
            load(path+"deportistaError.csv")

In [48]:
deportistaErrorDF.show()

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|  80|      199|
|            2|            A Lamusi|     1|  23|   170|  60|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|  NULL|NULL|      273|
|            4|Edgar Lindenau Aabye|     1|  34|  NULL|NULL|      278|
|            5|Christine Jacoba ...|     2|  21|   185|  82|      705|
|            6|     Per Knut Aaland|     1|  31|   188|  75|     1096|
|            7|        John Aalberg|     1|  31|   183|  72|     1096|
|            8|"Cornelia ""Cor""...|     2|  18|   168|NULL|      705|
|            9|    Antti Sami Aalto|     1|  26|   186|  96|      350|
|           10|"Einar Ferdinand ...|     1|  26|  NULL|NULL|      350|
|           11|  Jorma Ilmari Aalto|     1|  22|   182|76.5|      350|
|     

Si analizamos la columna altura nosotros queremos que sean enteros y en ausencia del valor deberia aparecer un null, aqui entran en el juego las **UDF**, las cuales trabajaran de forma nativa

In [49]:
from pyspark.sql.functions import udf

In [50]:
def conversionEnteros(valor):
    try:
        return int(valor) if len(valor) > 0 else 0
    except TypeError as e:
        return 0
    
conversionEnteros_udf = udf(lambda z: conversionEnteros(z), IntegerType())
sqlContext.udf.register("conversionEnteros_udf", conversionEnteros_udf)

deportistaErrorDF.select(conversionEnteros_udf("altura") \
                         .alias("alturaUDF")).show()

+---------+
|alturaUDF|
+---------+
|      180|
|      170|
|        0|
|        0|
|      185|
|      188|
|      183|
|      168|
|      186|
|        0|
|      182|
|      172|
|      159|
|      171|
|        0|
|      184|
|      175|
|      189|
|        0|
|      176|
+---------+
only showing top 20 rows



# DF y replicacion

In [51]:
from pyspark.storagelevel import StorageLevel

In [52]:
medallistaXAnio.is_cached

False

Cada vez que necesitamos el recurso Spark necesita "incovarlo" de nuevo, pero podemos indicarle a spark que lo guarde en cache.

In [53]:
# Creamos la persisttencia
medallistaXAnio.rdd.cache()

24/01/18 16:36:29 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , nombre_juego
 Schema: juego_id, anio
Expected: juego_id but found: 
CSV file: file:///home/carloscastau/Documents/platzi/data_engineer/curso-apache-spark-platzi-master/files/juegos.csv
24/01/18 16:36:29 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: evento_id, evento, deporte_id
 Schema: evento_id, nombre, deporte_id
Expected: nombre but found: evento
CSV file: file:///home/carloscastau/Documents/platzi/data_engineer/curso-apache-spark-platzi-master/files/evento.csv


MapPartitionsRDD[220] at javaToPython at NativeMethodAccessorImpl.java:0

https://spark.apache.org/docs/2.4.6/api/python/pyspark.html#pyspark.StorageLevel

In [54]:
# Saber el tipo de persistencia y nivel de replicacion
medallistaXAnio.rdd.getStorageLevel()

StorageLevel(False, True, False, False, 1)

In [55]:
# eliminando la persistencia para que no genere error
medallistaXAnio.rdd.unpersist()

MapPartitionsRDD[220] at javaToPython at NativeMethodAccessorImpl.java:0

In [56]:
# Usando persistencia MEMORY AND DISK con replicacion 2
medallistaXAnio.rdd.persist(StorageLevel.MEMORY_AND_DISK_2)

MapPartitionsRDD[220] at javaToPython at NativeMethodAccessorImpl.java:0

In [57]:
# Saber el tipo de persistencia y nivel de replicacion
medallistaXAnio.rdd.getStorageLevel()

StorageLevel(True, True, False, False, 2)

In [58]:
# eliminando la persistencia
medallistaXAnio.rdd.unpersist()

MapPartitionsRDD[220] at javaToPython at NativeMethodAccessorImpl.java:0

In [59]:
# Definiendo un StorageLevel custom
StorageLevel.MEMORY_AND_DISK_3 = StorageLevel(True, True, False, False, 3)

In [60]:
# Persistimos el DataFrame
medallistaXAnio.rdd.persist(StorageLevel.MEMORY_AND_DISK_3)

MapPartitionsRDD[220] at javaToPython at NativeMethodAccessorImpl.java:0

In [61]:
# Saber el tipo de persistencia y nivel de replicacion
medallistaXAnio.rdd.getStorageLevel()

StorageLevel(True, True, False, False, 3)

In [62]:
spark.stop()